In [1]:
!which python

/Users/apabook/Desktop/ds-protocol/static/tutorial/venv/bin/python


In [2]:
import json
import requests

# Complete Dataspace Flow


## Consts
Configuration of constant variables used throughout the notebook, including agent URLs and participant IDs.


In [3]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_provider_dsp_endpoint = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Dataspace setup
Authority is created and Consumer Agent and Provider Agent are registered in the Authority. Then, both participants onboard themselves in the system.


### Current Participants Information


In [4]:
url = data_space_provider + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:provider",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "saved_at": "2026-02-08T18:52:01.221800",
    "last_interaction": "2026-02-08T18:52:01.221834",
    "is_me": true
}


In [5]:
url = data_space_consumer + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:consumer",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "saved_at": "2026-02-08T18:52:03.933141",
    "last_interaction": "2026-02-08T18:52:03.933174",
    "is_me": true
}


### Provider and Consumer IDs
Credential exchange and token retrieval for API calls.


In [6]:
url = data_space_provider + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_consumer_participant_id:
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

In [7]:
url = data_space_consumer + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_provider_participant_id:
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

In [8]:
print("Provider DID: {}\n".format(data_space_provider_participant_id))
print("Provider token: {}\n".format(data_space_provider_participant_token))
print("Consumer DID: {}\n".format(data_space_consumer_participant_id))
print("Consumer token: {}".format(data_space_consumer_participant_token))

Provider DID: did:jwk:provider

Provider token: HzX-4U3Cw05--NXRJIoR1gxi3OTgfQ--EXQ2TUbF32c

Consumer DID: did:jwk:consumer

Consumer token: HzX-4U3Cw05--NXRJIoR1gxi3OTgfQ--EXQ2TUbF32c


## Create Dataset, Distributions, Policies and Connector Instances
Provider is going to create a catalog, a data service, a dataset and a distribution to be offered to the consumer. The consumer will then discover the offer and accept it.


In [9]:
url = data_space_provider + "/api/v1/catalog-agent/catalogs/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "dctConformsTo": null,
    "dctCreator": null,
    "dctIdentifier": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72",
    "dctIssued": "2026-02-08T18:52:23.541617Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainCatalog": true,
    "dspaceParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "foafHomePage": null,
    "id": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72"
}


In [10]:
url = data_space_provider + "/api/v1/catalog-agent/data-services/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_service_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72",
    "dcatEndpointDescription": null,
    "dcatEndpointUrl": "http://127.0.0.1:1200/dsp/current",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:data-service:1850f547-cf0c-4141-a908-1da7052ef78a",
    "dctIssued": "2026-02-08T18:52:23.622720Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainDataService": true,
    "id": "urn:data-service:1850f547-cf0c-4141-a908-1da7052ef78a"
}


In [11]:
url = data_space_provider + "/api/v1/catalog-agent/datasets"
payload = {
    "dctTitle": "My dataspace dataset",
    "catalogId": catalog_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a",
    "dctIssued": "2026-02-10T09:26:14.107661Z",
    "dctModified": null,
    "dctTitle": "My dataspace dataset",
    "id": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a"
}


In [12]:
url = data_space_provider + "/api/v1/catalog-agent/distributions"
payload = {
    "dctTitle": "Distribution of my dataset",
    "dctDescription": "This is the distribution of my dataset",
    "dcatAccessService": data_service_id,
    "datasetId": dataset_id,
    "dctFormats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "datasetId": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a",
    "dcatAccessService": "urn:data-service:1850f547-cf0c-4141-a908-1da7052ef78a",
    "dctDescription": "This is the distribution of my dataset",
    "dctFormat": "Http+Pull",
    "dctIssued": "2026-02-10T09:26:14.132968Z",
    "dctModified": null,
    "dctTitle": "Distribution of my dataset",
    "id": "urn:distribution:0a359095-1004-4f39-9c33-558ecd85d309"
}


## Create policies


In [13]:
url = data_space_provider + "/api/v1/catalog-agent/odrl-policies"
payload = {
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "rightOperand": "user",
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq"
                    }
                ]
            }

        ],
        "obligation": [],
        "prohibition": []
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-10T09:26:14.149250Z",
    "entity": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:e1d8b296-a4d2-464a-ab7e-5c38ef73f217",
    "instantiationParameters": null,
    "odrlOffer": {
        "obligation": [],
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            }
        ],
        "prohibition": []
    },
    "sourceTemplateId": null,
    "sourceTemplateVersion": null
}


## Create and configure connector to final system



In [14]:
url = data_space_provider + "/api/v1/connector/templates"
payload = {
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}"
        }
    },
    "parameters": [
        {
            "paramType": "STRING",
            "name": "ACCESS_URL",
            "title": "Url",
            "required": True
        },
        {
            "paramType": "VEC<STRING>",
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "required": True
        },
        {
            "paramType": "MAP<STRING,STRING>",
            "name": "ACCESS_HEADERS",
            "title": "Methods",
            "required": True
        }
    ]
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
connector_name = response_as_json["name"]
connector_version = response_as_json["version"]
print(json.dumps(response_as_json, indent=4))

{
    "name": "urn:connector-template:a92cc08a-ae38-424f-bde1-761291afe480",
    "author": "admin",
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-10T09:26:14.176800Z",
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}",
            "bodyTemplate": null
        }
    },
    "parameters": [
        {
            "name": "ACCESS_URL",
            "title": "Url",
            "description": null,
            "paramType": "STRING",
            "required": true,
            "defaultValue": null,
            "autoFilled": false,
            "autoFilledType": null
        },
        {
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "description": null,
            "paramType": "VEC<STRIN

In [15]:
url = data_space_provider + "/api/v1/connector/instances"
payload = {
    "templateName": connector_name,
    "templateVersion": connector_version,
    "distributionId": distribution_id,
    "metadata": {
        "description": "asasdasddas",
        "ownerId": "asdas"
    },
    "dryRun": False,
    "parameters": {
        "ACCESS_URL": "https://dummyjson.com/products",
        "ACCESS_METHODS": ["GET", "HEAD"],
        "ACCESS_HEADERS": {
            "Content-type": "application/json"
        }
    }
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "id": "urn:connector-instance:ae904410-6a7f-44d0-ab11-04b7bbf25392",
    "name": "urn:connector-template:a92cc08a-ae38-424f-bde1-761291afe480",
    "author": null,
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-10T09:26:14.203854Z",
    "authenticationConfig": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "https://dummyjson.com/products",
            "method": [
                "GET",
                "HEAD"
            ],
            "headers": {
                "Content-type": "application/json"
            },
            "bodyTemplate": null
        }
    },
    "distributionId": "urn:distribution:0a359095-1004-4f39-9c33-558ecd85d309"
}


## Consumer discovers the provider catalog and finds the offer


In [16]:
url = data_space_consumer + "/dsp/current/catalog/rpc/setup-catalog-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "filter": [], 
    "noCache": True
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["response"]["dataset"][-1]["hasPolicy"][0]["@id"]
target_id = response_as_json["response"]["dataset"][0]["@id"]
data_space_provider_dsp_endpoint = response_as_json["response"]["service"]["endpointURL"]
response_as_json


{'request': {'associatedAgentPeer': 'did:jwk:provider',
  'filter': [],
  'noCache': True},
 'response': {'@context': ['https://w3id.org/dspace/2025/1/context.jsonld'],
  '@type': 'Catalog',
  '@id': 'urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72',
  'identifier': 'urn:catalog:b67a04dd-2dfc-49b8-85aa-48f29deaca72',
  'issued': '2026-02-08T18:52:23.541617',
  'catalog': [],
  'dataset': [{'@type': 'Dataset',
    '@id': 'urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a',
    'identifier': 'urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a',
    'issued': '2026-02-10T09:26:14.107661',
    'title': 'My dataspace dataset',
    'hasPolicy': [{'@id': 'urn:odrl-policy:e1d8b296-a4d2-464a-ab7e-5c38ef73f217',
      'permission': [{'action': 'use',
        'constraint': [{'rightOperand': 'user',
          'leftOperand': 'did:web:hola.es',
          'operator': 'eq'}]}],
      'obligation': [],
      '@type': 'Offer',
      'prohibition': []}],
    'distribution': [{'@type': 'Distribution',
     

In [17]:
print("Selected dataset target: \n{}\n".format(target_id))
print("Selected policy id: \n{}\n".format(policy_id))
print("Provider negotiation URL for DSP 2025: \n{}\n".format(data_space_provider_dsp_endpoint))


Selected dataset target: 
urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a

Selected policy id: 
urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672

Provider negotiation URL for DSP 2025: 
http://127.0.0.1:1200/dsp/current



## Consumer discovers own callback address based on DSP version

In [18]:
url = data_space_consumer + "/.well-known/dspace-version/2025-1"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_dsp_endpoint = data_space_consumer + response_as_json["path"]
print("Consumer callback address: \n{}\n".format(data_space_consumer_dsp_endpoint))


Consumer callback address: 
http://127.0.0.1:1100/dsp/current



## Contract Negotiation

The process follows the negotiation protocol flow:
1. Request Init
2. Offer
3. Request
4. Acceptance
5. Verification
6. Finalization


In [19]:
# Initialization of negotiation request (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request-init"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"
            }
        ]
    }
}
try:
    response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
    response_as_json = response.json()
    cn_consumer_id = response_as_json["response"]["consumerPid"]
    cn_provider_id = response_as_json["response"]["providerPid"]
    print(json.dumps(response_as_json, indent=2))
except Exception as e:
    print("Error in response, {}.".format(e))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "use"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a"
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:66b00f2e-ac8e-4126-9574-c1002b6533ba",
    "state": "REQUESTED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http:

In [20]:
# Provider creates initial offer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,  # remove to test offer from provider
    "providerPid": cn_provider_id,  # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "supermegause"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a"
    },
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e481a885-d327-4de8-b292-e449d9bc2e37",
    "state": "OFFERED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.

In [21]:
# Consumer sends negotiation request based on offer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "use"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a"
    },
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:66b00f2e-ac8e-4126-9574-c1002b6533ba",
    "state": "REQUESTED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:120

In [22]:
# Provider updates/confirms the offer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:4b083039-3eb4-4158-84be-cf18be39f672",
      "permission": [
        {
          "action": "supermegause"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a"
    },
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e481a885-d327-4de8-b292-e449d9bc2e37",
    "state": "OFFERED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.

In [23]:
# Consumer accepts the offer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-acceptance"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "ACCEPTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:66b00f2e-ac8e-4126-9574-c1002b6533ba",
    "state": "ACCEPTED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T09:26:14.493820Z",
    "updatedAt": "2026-02-10T09:26:14.998134Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bc

In [24]:
# Provider creates the Agreement (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-agreement"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "AGREED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e481a885-d327-4de8-b292-e449d9bc2e37",
    "state": "AGREED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T09:26:14.374583Z",
    "updatedAt": "2026-02-10T09:26:15.100780Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-85

In [25]:
# Consumer verifies the agreement (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-verification"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "VERIFIED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:66b00f2e-ac8e-4126-9574-c1002b6533ba",
    "state": "VERIFIED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T09:26:14.493820Z",
    "updatedAt": "2026-02-10T09:26:15.192773Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bc

In [26]:
# Provider finalizes the negotiation (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-finalization"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    agreement = response_as_json
    agreement_id = response_as_json["negotiationAgentModel"]["agreement"]["id"]
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:570fa5fe-6a7e-4993-8e2c-e7c7314a35ba",
    "providerPid": "urn:provider-pid:b576e3bd-1c9f-4bcb-8577-a69999732606",
    "state": "FINALIZED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:e481a885-d327-4de8-b292-e449d9bc2e37",
    "state": "FINALIZED",
    "stateAttribute": null,
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T09:26:14.374583Z",
    "updatedAt": "2026-02-10T09:26:15.306692Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:b576e3bd-1c9f-4

In [27]:
print("Final agreement: \n{}\n".format(json.dumps(agreement["negotiationAgentModel"]["agreement"], indent=2)))
print("Final agreement id: \n{}\n".format(agreement_id))

Final agreement: 
{
  "id": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
  "negotiationAgentProcessId": "urn:negotiation-process:e481a885-d327-4de8-b292-e449d9bc2e37",
  "negotiationAgentMessageId": "urn:negotiation-message:b43a2100-165e-4c67-9bf6-a00e8691fd12",
  "consumerParticipantId": "",
  "providerParticipantId": "",
  "agreementContent": {
    "@id": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
    "@type": "Agreement",
    "assignee": "",
    "assigner": "",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "target": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a",
    "timestamp": "1770715575"
  },
  "target": "urn:dataset:1c05d2f6-b20b-432d-9e23-6560a885de1a",
  "state": "ACTIVE",
  "createdAt": "2026-02-10T09:26:15.115043Z",
  "updatedAt": "2026-02-10T09:26:15.340356Z"
}

Final agreement id: 
urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6



## Transfer Negotiation with Agents


In [28]:
# 1. Transfer request setup (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "agreementId": agreement_id,
    "format": "Http+Pull",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_process_consumer_pid = response_as_json["response"]["consumerPid"]
transfer_process_provider_pid = response_as_json["response"]["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "agreementId": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
    "format": "Http+Pull",
    "dataAddress": null,
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "state": "REQUESTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:5982e339-14e2-41e0-b67e-5056a5d84966",
    "state": "REQUESTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "Http+Pull",
    "agreementId": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
    "callbackAddress": "http://127.0.0.1:1200/dsp/curre

In [29]:
# 2. Start transfer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": {
        "@type": "DataAddress",
        "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
        "endpoint": "http://example.com",
        "endpointProperties": [
            {
                "@type": "EndpointProperty",
                "name": "authorization",
                "value": "TOKEN-ABCDEFG"
            },
            {
                "@type": "EndpointProperty",
                "name": "authType",
                "value": "bearer"
            }
        ]
    },
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "dataAddress": {
      "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
      "endpoint": "http://example.com",
      "endpointProperties": [
        {
          "name": "authorization",
          "value": "TOKEN-ABCDEFG"
        },
        {
          "name": "authType",
          "value": "bearer"
        }
      ]
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:9747470e-eb94-417f-835f-5f51697b49ca",
    "state": "STARTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "",
 

In [30]:
# 3. Suspend transfer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:5982e339-14e2-41e0-b67e-5056a5d84966",
    "state": "SUSPENDED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "Http+Pull",
    "agreementId": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": 

In [31]:
# 4. Restart transfer (Consumer -> Provider)
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "dataAddress": null
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:5982e339-14e2-41e0-b67e-5056a5d84966",
    "state": "STARTED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "Http+Pull",
    "agreementId": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T09:26:15.402

In [32]:
# 5. Suspension by Provider (Provider -> Consumer)
# (Note: Testing provider-initiated suspension)
url = data_space_provider + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:9747470e-eb94-417f-835f-5f51697b49ca",
    "state": "SUSPENDED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "Http+Pull",
    "agreementId": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": 

In [33]:
# Failure Test: Attempt start with invalid parameters
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": None
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "dataAddress": null
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "5000",
    "reason": [
      "HTTP Error 400 Bad Request: {\"@context\":[\"https://w3id.org/dspace/2025/1/context.jsonld\"],\"@type\":\"TransferError\",\"consumerPid\":null,\"providerPid\":null,\"code\":\"6030\",\"reason\":[\"TransferProcessMessageType TransferStartMessage is not allowed here. Current state is SUSPENDED ByProvider\",\"Failed to parse file\"]}"
    ]
  }
}


In [34]:
# Failure Test: Attempt duplicate or invalid suspension
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "6030",
    "reason": [
      "TransferProcessMessageType TransferSuspensionMessage is not allowed here. Current state is SUSPENDED",
      "Failed to parse file"
    ]
  }
}


In [35]:
# 6. Finalize transfer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/transfers/rpc/setup-completion"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:6cd362ff-9f71-407d-baee-cb17791707b8",
    "providerPid": "urn:provider-pid:b233662d-1b7c-462f-b46c-140bf0106ced",
    "state": "COMPLETED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:9747470e-eb94-417f-835f-5f51697b49ca",
    "state": "COMPLETED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "",
    "protocol": "DSP",
    "transferDirection": "Http+Pull",
    "agreementId": "urn:agreement:93ec68ff-e5c0-4e13-9568-41699ddc42d6",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-10T09:26:15.381546Z",
    "updatedAt